# ParentDocumentRetriever Tutorial

ParentDocumentRetriever를 활용하여 계층적 문서 검색을 구현하는 방법을 학습합니다.

## 📚 학습 목표
- ParentDocumentRetriever의 개념과 필요성 이해
- 문서 분할과 검색의 균형점 찾기
- 계층적 문서 구조를 활용한 효율적인 검색 구현

## 📋 주요 내용
1. ParentDocumentRetriever 개념
2. 전체 문서 검색 구현
3. 계층적 청크 크기 조절
4. 작은 청크로 검색하고 큰 청크로 반환하는 전략

## 1. Parent Document Retriever 개념

### 문서 검색과 문서 분할의 균형 잡기

문서 검색 과정에서 문서를 적절한 크기의 조각(청크)으로 나누는 것은 다음의 **상충되는 두 가지 중요한 요소를 고려**해야 합니다.

1. **작은 문서를 원하는 경우**: 이렇게 하면 문서의 임베딩이 그 의미를 가장 정확하게 반영할 수 있습니다. 문서가 너무 길면 임베딩이 의미를 잃어버릴 수 있습니다.
2. **각 청크의 맥락이 유지되도록 충분히 긴 문서를 원하는 경우**: 너무 작은 청크는 전체 맥락을 잃을 수 있습니다.

### `ParentDocumentRetriever`의 역할

이 두 요구 사항 사이의 균형을 맞추기 위해 `ParentDocumentRetriever`라는 도구가 사용됩니다. 이 도구는:
- 문서를 작은 조각으로 나누고, 이 조각들을 관리합니다
- 검색을 진행할 때는, 먼저 이 작은 조각들을 찾아냅니다
- 이 조각들이 속한 원본 문서(또는 더 큰 조각)의 식별자(ID)를 통해 전체적인 맥락을 파악합니다

여기서 '부모 문서'란, 작은 조각이 나누어진 원본 문서를 말합니다. 이는 전체 문서일 수도 있고, 비교적 큰 다른 조각일 수도 있습니다.

### 정리

- **문서 간의 계층 구조 활용**: `ParentDocumentRetriever`는 문서 검색의 효율성을 높이기 위해 문서 간의 계층 구조를 활용합니다.
- **검색 성능 향상**: 관련성 높은 문서를 빠르게 찾아내며, 주어진 질문에 대한 가장 적합한 답변을 제공하는 문서를 효과적으로 찾아낼 수 있습니다.

## 2. 환경 설정 및 필요 라이브러리 임포트

In [ ]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

In [1]:
from langchain.storage import InMemoryStore
from langchain_community.document_loaders import TextLoader
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.retrievers import ParentDocumentRetriever

## 3. 데이터 로드

In [2]:
loaders = [
    # 파일을 로드합니다.
    TextLoader("../appendix-keywords.txt"),
]

docs = []
for loader in loaders:
    # 로더를 사용하여 문서를 로드하고 docs 리스트에 추가합니다.
    docs.extend(loader.load())

## 4. 전체 문서 검색

이 모드에서는 전체 문서를 검색하고자 합니다. 따라서 `child_splitter` 만 지정하도록 하겠습니다.

- 나중에는 `parent_splitter` 도 지정하여 결과를 비교해 보겠습니다.

In [3]:
# 자식 분할기를 생성합니다.
child_splitter = RecursiveCharacterTextSplitter(chunk_size=200)

# DB를 생성합니다.
vectorstore = Chroma(
    collection_name="full_documents", embedding_function=OpenAIEmbeddings()
)

store = InMemoryStore()

# Retriever 를 생성합니다.
retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter,
)

### 문서 추가

`retriever.add_documents(docs, ids=None)` 함수로 문서목록을 추가합니다.

- `ids` 가 `None` 이면 자동으로 생성됩니다.
- `add_to_docstore=False` 로 설정시 document 를 중복으로 추가하지 않습니다. 단, 중복을 체크하기 위한 `ids` 값이 필수 값으로 요구됩니다.

In [4]:
# 문서를 검색기에 추가합니다. docs는 문서 목록이고, ids는 문서의 고유 식별자 목록입니다.
retriever.add_documents(docs, ids=None, add_to_docstore=True)

### 저장소 키 확인

이 코드는 추가된 문서의 키를 반환합니다.
- `store` 객체의 `yield_keys()` 메서드를 호출하여 반환된 키(key) 값들을 리스트로 변환합니다.

In [5]:
# 저장소의 모든 키를 리스트로 반환합니다.
list(store.yield_keys())

['7416183f-aade-4a33-bb75-709998516d08']

### 벡터스토어 직접 검색

In [6]:
# 유사도 검색을 수행합니다.
sub_docs = vectorstore.similarity_search("Word2Vec")

In [10]:
# sub_docs 리스트의 첫 번째 요소의 page_content 속성을 출력합니다.
print(sub_docs[0].page_content)

정의: Word2Vec은 단어를 벡터 공간에 매핑하여 단어 간의 의미적 관계를 나타내는 자연어 처리 기술입니다. 이는 단어의 문맥적 유사성을 기반으로 벡터를 생성합니다.
예시: Word2Vec 모델에서 "왕"과 "여왕"은 서로 가까운 위치에 벡터로 표현됩니다.
연관키워드: 자연어 처리, 임베딩, 의미론적 유사성


### ParentDocumentRetriever를 통한 검색

이제 전체 retriever에서 검색해 보겠습니다. 이 과정에서는 작은 청크(chunk)들이 위치한 **문서를 반환** 하기 때문에 상대적으로 큰 문서들이 반환될 것입니다.

In [11]:
# 문서를 검색하여 가져옵니다.
retrieved_docs = retriever.invoke("Word2Vec")

In [12]:
# 검색된 문서의 문서의 페이지 내용의 길이를 출력합니다.
print(
    f"문서의 길이: {len(retrieved_docs[0].page_content)}",
    end="\n\n=====================\n\n",
)

# 문서의 일부를 출력합니다.
print(retrieved_docs[0].page_content[2000:2500])

문서의 길이: 5733


 컴퓨팅을 도입하여 데이터 저장과 처리를 혁신하는 것은 디지털 변환의 예입니다.
연관키워드: 혁신, 기술, 비즈니스 모델

Crawling

정의: 크롤링은 자동화된 방식으로 웹 페이지를 방문하여 데이터를 수집하는 과정입니다. 이는 검색 엔진 최적화나 데이터 분석에 자주 사용됩니다.
예시: 구글 검색 엔진이 인터넷 상의 웹사이트를 방문하여 콘텐츠를 수집하고 인덱싱하는 것이 크롤링입니다.
연관키워드: 데이터 수집, 웹 스크래핑, 검색 엔진

Word2Vec

정의: Word2Vec은 단어를 벡터 공간에 매핑하여 단어 간의 의미적 관계를 나타내는 자연어 처리 기술입니다. 이는 단어의 문맥적 유사성을 기반으로 벡터를 생성합니다.
예시: Word2Vec 모델에서 "왕"과 "여왕"은 서로 가까운 위치에 벡터로 표현됩니다.
연관키워드: 자연어 처리, 임베딩, 의미론적 유사성
LLM (Large Language Model)

정의: LLM은 대규모의 텍스트 데이터로 훈련된 큰 규모의 언어 모델을


## 5. 더 큰 Chunk 의 크기를 조절

이전의 결과처럼 **전체 문서가 너무 커서 있는 그대로 검색하기에는 부적합** 할 수 있습니다.

이런 경우, 실제로 우리가 하고 싶은 것은:
1. 먼저 원시 문서를 더 큰 청크로 분할
2. 그 다음, 더 작은 청크로 분할
3. 작은 청크들을 인덱싱
4. 검색 시에는 더 큰 청크를 검색 (그러나 여전히 전체 문서는 아님)

- `RecursiveCharacterTextSplitter`를 사용하여 부모 문서와 자식 문서를 생성합니다.
  - 부모 문서는 `chunk_size`가 1000으로 설정되어 있습니다.
  - 자식 문서는 `chunk_size`가 200으로 설정되어 있으며, 부모 문서보다 작은 크기로 생성됩니다.

In [13]:
# 부모 문서를 생성하는 데 사용되는 텍스트 분할기입니다.
parent_splitter = RecursiveCharacterTextSplitter(chunk_size=1000)
# 자식 문서를 생성하는 데 사용되는 텍스트 분할기입니다.
# 부모보다 작은 문서를 생성해야 합니다.
child_splitter = RecursiveCharacterTextSplitter(chunk_size=200)
# 자식 청크를 인덱싱하는 데 사용할 벡터 저장소입니다.
vectorstore = Chroma(
    collection_name="split_parents", embedding_function=OpenAIEmbeddings()
)
# 부모 문서의 저장 계층입니다.
store = InMemoryStore()

### 계층적 ParentDocumentRetriever 초기화

`ParentDocumentRetriever`를 초기화하는 코드입니다.

- `vectorstore` 매개변수는 문서 벡터를 저장하는 벡터 저장소를 지정합니다.
- `docstore` 매개변수는 문서 데이터를 저장하는 문서 저장소를 지정합니다.
- `child_splitter` 매개변수는 하위 문서를 분할하는 데 사용되는 문서 분할기를 지정합니다.
- `parent_splitter` 매개변수는 상위 문서를 분할하는 데 사용되는 문서 분할기를 지정합니다.

`ParentDocumentRetriever`는 계층적 문서 구조를 처리하며, 상위 문서와 하위 문서를 별도로 분할하고 저장합니다. 이를 통해 검색 시 상위 문서와 하위 문서를 효과적으로 활용할 수 있습니다.

In [14]:
retriever = ParentDocumentRetriever(
    # 벡터 저장소를 지정합니다.
    vectorstore=vectorstore,
    # 문서 저장소를 지정합니다.
    docstore=store,
    # 하위 문서 분할기를 지정합니다.
    child_splitter=child_splitter,
    # 상위 문서 분할기를 지정합니다.
    parent_splitter=parent_splitter,
)

In [15]:
retriever.add_documents(docs)  # 문서를 retriever에 추가합니다.

In [16]:
# 저장소에서 키를 생성하고 리스트로 변환한 후 길이를 반환합니다.
len(list(store.yield_keys()))

7

### 벡터 저장소가 여전히 작은 청크를 검색하는지 확인

In [17]:
# 유사도 검색을 수행합니다.
sub_docs = vectorstore.similarity_search("Word2Vec")
# sub_docs 리스트의 첫 번째 요소의 page_content 속성을 출력합니다.
print(sub_docs[0].page_content)

정의: Word2Vec은 단어를 벡터 공간에 매핑하여 단어 간의 의미적 관계를 나타내는 자연어 처리 기술입니다. 이는 단어의 문맥적 유사성을 기반으로 벡터를 생성합니다.
예시: Word2Vec 모델에서 "왕"과 "여왕"은 서로 가까운 위치에 벡터로 표현됩니다.
연관키워드: 자연어 처리, 임베딩, 의미론적 유사성


### 계층적 Retriever를 통한 검색

이번에는 `retriever` 객체의 `invoke()` 메서드를 사용하여 문서를 검색합니다.
작은 청크로 검색하되, 해당 청크가 속한 더 큰 부모 청크를 반환합니다.

In [18]:
# 문서를 검색하여 가져옵니다.
retrieved_docs = retriever.invoke("Word2Vec")

# 검색된 문서의 첫 번째 문서의 페이지 내용의 길이를 반환합니다.
print(retrieved_docs[0].page_content)

정의: 트랜스포머는 자연어 처리에서 사용되는 딥러닝 모델의 한 유형으로, 주로 번역, 요약, 텍스트 생성 등에 사용됩니다. 이는 Attention 메커니즘을 기반으로 합니다.
예시: 구글 번역기는 트랜스포머 모델을 사용하여 다양한 언어 간의 번역을 수행합니다.
연관키워드: 딥러닝, 자연어 처리, Attention

HuggingFace

정의: HuggingFace는 자연어 처리를 위한 다양한 사전 훈련된 모델과 도구를 제공하는 라이브러리입니다. 이는 연구자와 개발자들이 쉽게 NLP 작업을 수행할 수 있도록 돕습니다.
예시: HuggingFace의 Transformers 라이브러리를 사용하여 감정 분석, 텍스트 생성 등의 작업을 수행할 수 있습니다.
연관키워드: 자연어 처리, 딥러닝, 라이브러리

Digital Transformation

정의: 디지털 변환은 기술을 활용하여 기업의 서비스, 문화, 운영을 혁신하는 과정입니다. 이는 비즈니스 모델을 개선하고 디지털 기술을 통해 경쟁력을 높이는 데 중점을 둡니다.
예시: 기업이 클라우드 컴퓨팅을 도입하여 데이터 저장과 처리를 혁신하는 것은 디지털 변환의 예입니다.
연관키워드: 혁신, 기술, 비즈니스 모델

Crawling

정의: 크롤링은 자동화된 방식으로 웹 페이지를 방문하여 데이터를 수집하는 과정입니다. 이는 검색 엔진 최적화나 데이터 분석에 자주 사용됩니다.
예시: 구글 검색 엔진이 인터넷 상의 웹사이트를 방문하여 콘텐츠를 수집하고 인덱싱하는 것이 크롤링입니다.
연관키워드: 데이터 수집, 웹 스크래핑, 검색 엔진

Word2Vec

정의: Word2Vec은 단어를 벡터 공간에 매핑하여 단어 간의 의미적 관계를 나타내는 자연어 처리 기술입니다. 이는 단어의 문맥적 유사성을 기반으로 벡터를 생성합니다.
예시: Word2Vec 모델에서 "왕"과 "여왕"은 서로 가까운 위치에 벡터로 표현됩니다.
연관키워드: 자연어 처리, 임베딩, 의미론적 유사성
LLM (Large Language Model)


## 6. 요약 및 핵심 포인트

### ParentDocumentRetriever의 장점

1. **검색 정확도 향상**: 작은 청크로 정확한 의미 매칭
2. **맥락 보존**: 큰 청크 반환으로 전체 맥락 유지
3. **유연한 구조**: 다양한 크기의 parent/child splitter 조합 가능

### 주요 사용 사례

- 긴 문서에서 특정 정보를 찾되, 전체 맥락이 필요한 경우
- 기술 문서, 법률 문서 등 세부 내용과 전체 맥락이 모두 중요한 경우
- RAG 시스템에서 검색 품질을 향상시키고자 할 때

### 실습 결과 비교

| 구분 | child_splitter만 사용 | parent_splitter 추가 사용 |
|------|---------------------|------------------------|
| 반환 문서 크기 | 전체 문서 (5733자) | 부모 청크 (약 1000자) |
| 검색 정확도 | 높음 | 높음 |
| 맥락 정보 | 과도함 | 적절함 |
| 처리 효율성 | 낮음 | 높음 |